<a href="https://colab.research.google.com/github/AlvinKimata/Video-recognition-and-classification/blob/main/3D_CNN_on_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'https://debonair-sa.s3.af-south-1.amazonaws.com/datasets/3D+MNIST.zip'

--2022-09-27 12:24:10--  https://debonair-sa.s3.af-south-1.amazonaws.com/datasets/3D+MNIST.zip
Resolving debonair-sa.s3.af-south-1.amazonaws.com (debonair-sa.s3.af-south-1.amazonaws.com)... 52.95.175.41
Connecting to debonair-sa.s3.af-south-1.amazonaws.com (debonair-sa.s3.af-south-1.amazonaws.com)|52.95.175.41|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160210751 (153M) [application/zip]
Saving to: ‘3D+MNIST.zip’

3D+MNIST.zip        100%[===================>] 152.79M  12.3MB/s    in 15s     

2022-09-27 12:24:26 (10.5 MB/s) - ‘3D+MNIST.zip’ saved [160210751/160210751]



In [ ]:
import os
import h5py
import zipfile
import numpy as np

import tensorflow as tf

In [ ]:
#Extract the dataset.
zip_ref = zipfile.ZipFile('/content/3D+MNIST.zip')
zip_ref.extractall()
zip_ref.close()

## Load the dataset from the `full_dataset_vectors.h5' file.`

In [ ]:
with h5py.File('/content/full_dataset_vectors.h5', 'r') as dataset:
  X_train, X_test = dataset['X_train'][:], dataset['X_test'][:]
  y_train, y_test = dataset['y_train'][:], dataset['y_test'][:]


X_train = np.array(X_train)
X_test = np.array(X_test)

print(f'Train shape is: {X_train.shape}')
print(f'Test shape is: {X_test.shape}')

Train shape is: (10000, 4096)
Test shape is: (2000, 4096)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], 16, 16, 16, 1)
X_test = X_test.reshape(X_test.shape[0], 16, 16, 16, 1)
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes = 10)

In [ ]:
X_train.shape[1:]

(16, 16, 16, 1)

In [ ]:
#Building the 3D CNN.
model = tf.keras.Sequential([
    tf.keras.layers.Conv3D(32, (3, 3, 3), activation = 'relu', input_shape = (X_train.shape[1:]), bias_initializer = tf.keras.initializers.Constant(0.01)),
    tf.keras.layers.Conv3D(32, (3, 3, 3), activation = 'relu', bias_initializer = tf.keras.initializers.Constant(0.01)),
    tf.keras.layers.MaxPooling3D(pool_size = (2, 2, 2)),

    tf.keras.layers.Conv3D(64, (3, 3, 3), activation = 'relu'),
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling3D(pool_size = (2, 2, 2)),

    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation = 'softmax')

])


#Compile the model.
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = 'categorical_crossentropy',
              metrics = 'accuracy')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 14, 14, 14, 32)    896       
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 12, 12, 32)    27680     
                                                                 
 max_pooling3d (MaxPooling3D  (None, 6, 6, 6, 32)      0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 4, 4, 4, 64)       55360     
                                                                 
 conv3d_3 (Conv3D)           (None, 2, 2, 2, 64)       110656    
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 1, 1, 1, 64)      0         
 3D)                                                    

In [ ]:
model.fit(X_train, y_train, epochs = 200, batch_size = 128, 
          verbose = 1, validation_data = (X_test, y_test),
          callbacks = tf.keras.callbacks.EarlyStopping(patience = 45))

Epoch 1/200
79/79 [==============================] - 15s 35ms/step - loss: 2.3026 - accuracy: 0.1052 - val_loss: 2.2994 - val_accuracy: 0.1260
Epoch 2/200
79/79 [==============================] - 2s 30ms/step - loss: 2.2618 - accuracy: 0.1286 - val_loss: 2.1023 - val_accuracy: 0.2505
Epoch 3/200
79/79 [==============================] - 2s 29ms/step - loss: 2.0640 - accuracy: 0.2421 - val_loss: 1.8202 - val_accuracy: 0.3365
Epoch 4/200
79/79 [==============================] - 2s 30ms/step - loss: 1.8650 - accuracy: 0.3248 - val_loss: 1.5820 - val_accuracy: 0.4210
Epoch 5/200
79/79 [==============================] - 2s 29ms/step - loss: 1.7435 - accuracy: 0.3716 - val_loss: 1.4833 - val_accuracy: 0.4990
Epoch 6/200
79/79 [==============================] - 2s 29ms/step - loss: 1.6406 - accuracy: 0.4046 - val_loss: 1.3705 - val_accuracy: 0.5240
Epoch 7/200
79/79 [==============================] - 2s 31ms/step - loss: 1.5757 - accuracy: 0.4304 - val_loss: 1.3672 - val_accuracy: 0.5345
Epoch

In [ ]:
#Preform inference.
metrics = np.array(model.evaluate(X_test, y_test))
print(metrics * 100)

63/63 [==============================] - 0s 5ms/step - loss: 1.1211 - accuracy: 0.6895
[112.11003065  68.94999743]
